## 유방암 데이터로 실습하기

데이터셋 정보
Breast Cancer Wisconsin Dataset

샘플 수: 569개 (악성 212개, 양성 357개)

특성 수: 30개 (종양의 크기, 모양, 질감 등)

태스크: 종양이 악성인지 양성인지 분류

In [ ]:
import pandas as pd

In [8]:
# 데이터 불러오기
data = load_breast_cancer()
X, y = data.data, data.target
# 데이터 분리 (훈련:테스트 = 8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# 참고: Stratify를 사용하여 y의 클래스 비율(악성/양성)을 훈련 세트와 테스트 세트가 동일하게 유지하도록 나눈다.

print(f"훈련 세트 크기: {X_train.shape}, 테스트 세트 크기: {X_test.shape}")

훈련 세트 크기: (455, 30), 테스트 세트 크기: (114, 30)


#### 과제 1: Optimizer 비교
SGD, Momentum, Adam으로 학습 비교

의료 데이터에서 어떤 Optimizer가 좋은지 확인

False Negative(악성을 양성으로) 최소화가 중요

In [ ]:
# 1. 모델 정의
# 초기 가중치를 동일하게 설정하여 비교의 공정성을 확보합니다.
# hidden_layer_sizes는 초보 단계에서는 간단하게 100으로 고정합니다.

# 순수 SGD (momentum=0.0)
sgd_model = MLPClassifier(
    hidden_layer_sizes=(100,), 
    solver='sgd', 
    learning_rate_init=0.01, 
    momentum=0.0, 
    max_iter=1000, 
    random_state=42
)

# Momentum (momentum=0.9)
momentum_model = MLPClassifier(
    hidden_layer_sizes=(100,), 
    solver='sgd', 
    learning_rate_init=0.01, 
    momentum=0.9, 
    max_iter=1000, 
    random_state=42
)

# Adam (solver='adam')
adam_model = MLPClassifier(
    hidden_layer_sizes=(100,), 
    solver='adam', 
    learning_rate_init=0.001, # Adam의 일반적인 추천 학습률 [cite: 158]
    max_iter=1000, 
    random_state=42
)

models = {
    "Pure SGD": sgd_model,
    "Momentum": momentum_model,
    "Adam": adam_model
}

results = []

# 2. 학습 및 평가
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] 

    # FN (False Negative) 계산 및 평가 지표 수집
    cm = confusion_matrix(y_test, y_pred)
    # TN: 0을 0으로, FP: 0을 1로, FN: 1을 0으로, TP: 1을 1로
    # 참고: 유방암 데이터셋에서 악성은 0, 양성은 1입니다. 
    # 따라서 악성(0)을 양성(1)으로 잘못 분류하는 것이 FN입니다.
    # 하지만 MLPClassifier의 confusion_matrix는 클래스 인덱스를 따르므로 
    # 클래스 0(악성)에 대한 FN을 찾아야 합니다.
    
    # 클래스 0 (악성)을 1 (양성)으로 예측한 경우 (FP)를 
    # **의료적 맥락**에서는 FN(악성인데 놓침)으로 간주합니다.
    # 여기서 악성(0)은 Negative, 양성(1)은 Positive입니다.
    
    # 헷갈리실 수 있으니, 의료 맥락의 FN(실제 악성인데 양성으로 예측)을 구하려면
    # 실제 악성(0)이면서 예측 양성(1)인 값, 즉 FP를 찾아서 FN_Count로 사용합니다.
    
    # cm: [[TN, FP], [FN, TP]]
    tn, fp, fn, tp = cm.ravel()
    
    results.append({
        "Optimizer": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_prob),
        # 의료 맥락의 FN (실제 악성을 놓친 수) = FP (실제 0을 1로 예측)
        "FN_Count (의료적 위험)": fp, 
        "TN": tn, "FP": fp, "FN": fn, "TP": tp
    })

results_df = pd.DataFrame(results).sort_values(by='FN_Count (의료적 위험)')

In [ ]:
print("### 📊 Optimizer 비교 결과 (MLPClassifier 사용)")
print("FN_Count (의료적 위험): 실제 악성(0)을 양성(1)으로 잘못 예측한 수입니다. 이 값이 가장 작아야 합니다.\n")
print(results_df.to_markdown(index=False, floatfmt=".4f"))

#### 과제 2: 하이퍼파라미터 튜닝
GridSearchCV vs RandomSearchCV 비교

LogisticRegression의 최적 파라미터 찾기

정확도와 AUC 모두 확인

시간 효율성 비교